## 01_Mixtral_Mini_Dev_Set
In this notebook, we will load the mini development dataset, and use Mixtral 8x7B model in Colab A100 GPU to process (summarize, simplify) the text

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
# load mini dev set for elife
import pandas as pd
import io

# df = pd.read_json("./drive/MyDrive/531/milestone5/data/mini_dev_set/eLife_val.jsonl",
#                   orient="records",
#                   lines=True)

df = pd.read_json("./drive/MyDrive/531/milestone5/data/mini_dev_set/PLOS_val.jsonl",
                  orient="records",
                  lines=True)

# df = pd.read_json("./data/eLife_val.jsonl",
#                   orient="records",
#                   lines=True)

df.head()

,lay_summary,article,headings,keywords,id
0,"Yersinia pestis , the bacterial agent of plagu...",Fleas can transmit Yersinia pestis by two mech...,"[Abstract, Introduction, Results, Discussion, ...","[united states, invertebrates, medicine and he...",journal.ppat.1006859
1,The genome of all vertebrates is heavily colon...,Endogenous retroviruses ( ERVs ) are remnants ...,"[Abstract, Introduction, Results, Discussion, ...","[viruses, sheep, virology]",journal.ppat.0030170
2,The molecular mechanisms underlying directed c...,The Drosophila embryonic gonad is assembled fr...,"[Abstract, Introduction, Results, Discussion, ...",[],journal.pgen.1003720
3,Contrary to the long-standing belief that no n...,"Recently , we presented a study of adult neuro...","[Abstract, Introduction, Model, Results, Discu...",[computational biology/computational neuroscie...,journal.pcbi.1001063
4,Embryonic stem cells have two remarkable prope...,Understanding the transcriptional regulation o...,"[Abstract, Introduction, Results, Discussion, ...","[developmental biology, cell biology, mammals,...",journal.pgen.0030145


In [18]:
# take some item to inspect
k = 5
item = df.iloc[k]
print("Lay summary = ", item["lay_summary"])
print("Full text article =", item["article"])

Lay summary =  The current model of HCV egress is that virions assemble at lipid droplets , envelope at the ER and then likely exit the hepatocyte via the secretory pathway in association with apolipoproteins . To gain a more detailed insight into infectious HCV release , we combined an RNAi analysis of host factors that are required for infectious HCV secretion with live cell imaging of HCV core trafficking . Using this approach , we identified numerous components of the secretory pathway that are both required for infectious HCV release and co-traffic with HCV core . The dynamics of HCV core trafficking , both in terms of frequency of transport , particle velocity , and the corresponding run lengths were quantified . We observe that dynamic core movements in the periphery require NS2 , a viral protein required for virion assembly . Core co-traffics with multiple components of the secretory pathway , including the Golgi , recycling endosome , microtubules , VAMP1 secretory vesicles , 

In [19]:
def build_text_with_headings(item):
  """
    return the article text with heading embedded
  """
  result = ""
  paras = item["article"].split("\n")
  headings = item["headings"]
  if len(paras) != len(headings):
    print("Error, not matching length")
    return item["article"]
  else:
    for (heading, paragraph) in zip(headings, paras):
      result += f"## {heading}\n{paragraph}\n\n"

  return result


In [20]:
print(build_text_with_headings(item))

## Abstract
The current model of hepatitis C virus ( HCV ) production involves the assembly of virions on or near the surface of lipid droplets , envelopment at the ER in association with components of VLDL synthesis , and egress via the secretory pathway . However , the cellular requirements for and a mechanistic understanding of HCV secretion are incomplete at best . We combined an RNA interference ( RNAi ) analysis of host factors for infectious HCV secretion with the development of live cell imaging of HCV core trafficking to gain a detailed understanding of HCV egress . RNAi studies identified multiple components of the secretory pathway , including ER to Golgi trafficking , lipid and protein kinases that regulate budding from the trans-Golgi network ( TGN ) , VAMP1 vesicles and adaptor proteins , and the recycling endosome . Our results support a model wherein HCV is infectious upon envelopment at the ER and exits the cell via the secretory pathway . We next constructed infectiou

In [6]:
!pip install bitsandbytes

In [7]:
!pip install transformers
!pip install accelerate

In [8]:
!pip install  flash_attn

In [9]:
# load Mixtral 8x7B, 4-bit
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1" # this probably works on A100
tokenizer = AutoTokenizer.from_pretrained(model_id)

# the following WORKS with V100, and T4 but very slow without 4_bit
# model = AutoModelForCausalLM.from_pretrained(model_id,
#                                              load_in_4bit=True,
#                                              device_map="auto")

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             load_in_4bit=True,
                                             torch_dtype=torch.float16,
                                             use_flash_attention_2=True,
                                             device_map="auto")  # flash attention



The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [10]:
# test if the model works correctly
messages = [
    {"role": "user", "content": "What is the capital city of British Columbia? Answer in 1 sentence"},
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

outputs = model.generate(input_ids, max_new_tokens=1000)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:388: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slo

[INST] What is the capital city of British Columbia? Answer in 1 sentence [/INST] The capital city of British Columbia, a province in Canada, is Victoria.


In [11]:
# filter out the text between [INST] ... [/INST]
# text = tokenizer.decode(outputs[0], skip_special_tokens=True)
s = "[INST] What is the capital city of British Columbia? Answer in 1 sentence [/INST] The capital city of British Columbia, a province in Canada, is Victoria."
id = s.find('[/INST]') + len('[/INST]')
result = s[id:]
print(result)

 The capital city of British Columbia, a province in Canada, is Victoria.


In [21]:
# test with new prompt
# max_char = 20000
# prompt = f"Simplify and summarize in minimum 250 to maximum 500 words, combine answer into 1 paragraph, keep important factual details. Also include key facts in 1-5 concise sentences: {s[:max_char]}"
# prompt = f"Simplify and summarize in 200 to 300 words:  {s[:max_char]}"

# def summarize(article, max_char=20000):
def summarize(article, max_char=-1):
  """
    summarize the article text, with
  """
  print("Processing text =", article[:100])
  print("---------------")
  lay_summary_example = "Over 90% of adults around the world are infected with the Epstein-Barr virus . Like other closely related viruses , such as those that cause chicken pox and cold sores , an infection lasts for the rest of the person’s life , although the virus generally remains in a latent or dormant state . However , under certain conditions the latent viruses can cause cancers to develop; in fact , it is estimated that such infections are responsible for nearly 2% of all cancer deaths worldwide . One way that healthy human cells prevent cancer is by triggering their own death in a process called apoptosis . The Epstein-Barr virus can block apoptosis , therefore making the cells more likely to become cancerous . Previous research identified one protein in the Epstein-Barr virus that promotes cancer by preventing infected cells from dying as normal . However , even in the absence of this protein , Epstein-Barr virus-infected cells remain resistant to apoptosis . This suggests that the virus has another way of blocking cell death . Price et al . have now used a technique that stresses living cells in a way that reveals which proteins prevent apoptosis to study human cells infected with the Epstein-Barr virus . This revealed that soon after infection , the virus could force the human cell to produce MCL-1 , a protein that prevents cell death . Later , the Epstein-Barr virus enlisted a second human protein called BFL-1 , which makes the infected cell further resistant to apoptosis . Price et al . discovered that a protein in the Epstein-Barr virus called EBNA3A controls the production of the MCL-1 and BFL-1 proteins . In the future , developing therapies that target these proteins may lead to new treatments for cancers caused by the Epstein-Barr virus . Such treatments would be likely to have fewer side effects for patients than traditional chemotherapies ."
  prompt = f"The following text is from a medical research paper, which contains medical terms. The section headings have ## tag before it. Pay special attention to 'Abstract', 'Introduction' and 'Conclusion' sections. Simplify and summarize in minimum 250 to maximum 500 words, combine answer into 1 paragraph, keeping important facts. One good example of lay summary is '{lay_summary_example}'. Now do a lay summary on this text: {article[:max_char]}"
  messages = [
      {"role": "user", "content": prompt},
  ]

  input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

  outputs = model.generate(input_ids, max_new_tokens=1000)
  # print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  id = output_text.find('[/INST]') + len('[/INST]')
  result = output_text[id:]

  return result


In [22]:
# test function
text = build_text_with_headings(item)
print("--------------")
print("Summary =\n", summarize(text))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


--------------
Processing text = ## Abstract
The current model of hepatitis C virus ( HCV ) production involves the assembly of virio
---------------
Summary =
  Hepatitis C virus (HCV) is a member of the Flaviviridae family that causes chronic liver infections in humans. The virus has a single-stranded positive RNA genome that encodes for structural and non-structural proteins. The structural proteins include core and the glycoproteins E1 and E2, while the non-structural proteins include p7, NS2, NS3, NS4A, NS4B, NS5A, and NS5B. The virus infects hepatocytes in the liver and uses several host factors for entry, replication, and egress.

A recent study combined RNA interference (RNAi) analysis and live cell imaging to understand the cellular requirements and mechanisms of HCV egress. The RNAi analysis identified multiple components of the secretory pathway, including ER to Golgi trafficking, lipid and protein kinases, and the recycling endosome, that are required for HCV egress. The st

In [23]:
# create empty column
df["mixtral_summary"] = df["article"].apply(lambda x:"")
df.head()

n = len(df)
max_row = 50 # for dev. Set to n for full set

for i in range(max_row):
  print("Processing item i =", i)
  item = df.iloc[i]
  text = build_text_with_headings(item)
  summary = summarize(text)
  print("Summary =", summary[:200])
  # df.at[i, "llm_templates"] = parsed_item
  df.at[i, "mixtral_summary"] = summary
  print("------------------------")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing item i = 0
Processing text = ## Abstract
Fleas can transmit Yersinia pestis by two mechanisms , early-phase transmission ( EPT ) 
---------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summary =  Fleas can transmit the bacteria Yersinia pestis, which causes plague, through two methods: early-phase transmission (EPT) and biofilm-dependent transmission (BDT). EPT occurs when fleas feed on an in
------------------------
Processing item i = 1
Processing text = ## Abstract
Endogenous retroviruses ( ERVs ) are remnants of ancient retroviral infections of the ho
---------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Summary =  Burkholderia pseudomallei is a bacterium that causes melioidosis, a serious disease with high mortality rates even with proper diagnosis and treatment. The bacterium is considered an emerging pathoge
------------------------
Processing item i = 47
Processing text = ## Abstract
Offspring of Schistosoma mansoni-infected women in schistosomiasis-endemic areas may be 
---------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summary =  Schistosomiasis is a parasitic infection that can affect pregnant women and their unborn children. A study in Uganda examined the effects of praziquantel treatment of Schistosoma mansoni during pregn
------------------------
Processing item i = 48
Processing text = ## Abstract
Hox proteins play fundamental roles in controlling morphogenetic diversity along the ant
---------------


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Summary =  The research paper is about the regulation of the cell death gene, reaper (rpr), by the Hox protein, Deformed (Dfd), in Drosophila. The researchers found that Dfd regulates rpr expression in a highly
------------------------
Processing item i = 49
Processing text = ## Abstract
The antiproliferative response to anticancer treatment is the result of concurrent respo
---------------
Summary =  This research uses time-lapse live cell microscopy and flow cytometry to study how cancer cells respond to X-ray treatment. The researchers built a computational model to simulate the process of prol
------------------------


In [24]:
# write output
# filename = "./data/eLife_val_output.csv"
filename = "./data/plos_val_output.csv"

df.to_csv(filename,
          index=False)

In [30]:
# write txt file
filename_txt = "./data/plos.txt"

# clean text -> 1 paragraph
df["txt_summary"] = df["mixtral_summary"].apply(lambda text: (text
                                                              .replace("\n", "")
                                                              .strip('"')
                                                              .strip()
                                                              ))

df["txt_summary"].to_csv(filename_txt,
                         index=False,
                         header=False,
                         sep="\n"

                         )